In [1]:
import pandas as pd
import numpy as np
import math

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.metrics import accuracy_score
from scipy.stats import mode
import pandas as pd
import numpy as np
import errno
import copy
from copy import deepcopy
import csv
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
def output_rates(input_data, output_data, attribute_name, privileged=None, unprivileged=None, favourable=None, unfavourable=None):

    index_attribute = input_data.feature_names.index(attribute_name)
    privileged = float(privileged)
    unprivileged = float(unprivileged)
    
    input_priv = input_data.labels[np.where(input_data.features[:,index_attribute] == privileged)]
    output_priv = output_data.labels[np.where(output_data.features[:,index_attribute] == privileged)]
    priv_labels = np.concatenate((input_priv, output_priv), axis=1)
    
    input_unpriv = input_data.labels[np.where(input_data.features[:,index_attribute] == unprivileged)]
    output_unpriv = output_data.labels[np.where(output_data.features[:,index_attribute] == unprivileged)]
    unpriv_labels = np.concatenate((input_unpriv, output_unpriv), axis=1)
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    for i in range(priv_labels.shape[0]):
        input_label = priv_labels[i][0]
        output_label = priv_labels[i][1]
        if input_label == output_label:
            if input_label == unfavourable:
                tn = tn + 1
            else:
                tp = tp + 1
        else:
            if input_label == favourable and output_label == unfavourable:
                fn = fn + 1
            else:
                fp = fp + 1
    
    rates_privileged = [tp,fp,tn,fn]
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    for i in range(unpriv_labels.shape[0]):
        input_label = unpriv_labels[i][0]
        output_label = unpriv_labels[i][0]
        if input_label == output_label:
            if input_label == unfavourable:
                tn = tn + 1
            else:
                tp = tp + 1
        else:
            if input_label == favourable and output_label == unfavourable:
                fn = fn + 1
            else:
                fp = fp + 1
                
    rates_unprivileged = [tp,fp,tn,fn]           
    
    rates_list = [rates_privileged, rates_unprivileged]
    
    return rates_list

In [ ]:
# input privileged and unprivileged as integers
# input favourable and unfavourable as integers
# data as a pandas dataframe
# attribute_name as string

def statistical_parity_diff(data, attribute_name, privileged=None, unprivileged=None, favourable=None, unfavourable=None):
    protected_counts = data[attribute_name].value_counts().to_dict()
    favourable_outcomes_privileged_group = data.loc[data[attribute_name] == privileged]
    ratio_privileged = favourable_outcomes_privileged_group.iloc[:,-1].value_counts().to_dict()
    print(ratio_privileged)
    if favourable in ratio_privileged.keys():
        ratio_privileged = ratio_privileged[favourable]/favourable_outcomes_privileged_group.shape[0]
    else:
        ratio_privileged = 0
    print(ratio_privileged)
    favourable_outcomes_unprivileged_group = data.loc[data[attribute_name] == unprivileged]
    ratio_unprivileged = favourable_outcomes_unprivileged_group.iloc[:,-1].value_counts().to_dict()
    print(ratio_unprivileged)
    if favourable in ratio_unprivileged.keys():
        ratio_unprivileged = ratio_unprivileged[favourable]/favourable_outcomes_unprivileged_group.shape[0]
    else:
        ratio_unprivileged = 0
    print(ratio_unprivileged)

    fairness = ratio_privileged - ratio_unprivileged
    
    return fairness

In [156]:
# input privileged and unprivileged as integers
# input favourable and unfavourable as integers
# data as a pandas dataframe
# attribute_name as string

def disparate_impact(data, attribute_name, privileged=None, unprivileged=None, favourable=None, unfavourable=None):
    protected_counts = data[attribute_name].value_counts().to_dict()
    favourable_outcomes_privileged_group = data.loc[data[attribute_name] == privileged]
    
    favourable_outcomes_unprivileged_group = data.loc[data[attribute_name] == unprivileged]
    ratio_unprivileged = favourable_outcomes_unprivileged_group.iloc[:,-1].value_counts().to_dict()
    print(ratio_unprivileged)
    if favourable in ratio_unprivileged.keys():
        ratio_unprivileged = ratio_unprivileged[favourable]/favourable_outcomes_unprivileged_group.shape[0]
    else:
        ratio_unprivileged = 0
        return 0
    print("ratio_unpr: ",ratio_unprivileged)
    
    ratio_privileged = favourable_outcomes_privileged_group.iloc[:,-1].value_counts().to_dict()
    print(ratio_privileged)
    if favourable in ratio_privileged.keys():
        ratio_privileged = ratio_privileged[favourable]/favourable_outcomes_privileged_group.shape[0]
    else:
        ratio_privileged = 0
        return math.inf
    print("ratio_priv: ",ratio_privileged)
    
    fairness = ratio_unprivileged/ratio_privileged
    
    return fairness

In [ ]:
def equal_opp_diff(input_data, output_data, attribute_name, privileged, unprivileged, favourable, unfavourable):
    outcome_both = output_rates(input_data, output_data, attribute_name, privileged, unprivileged, favourable, unfavourable)
    
    # [tp, fp, tn, fn]
    outcome_privileged = rates_both[0]
    outcome_unprivileged = rates_both[1]
    
    # true positive rate = tp / (tp + fn)
    tpr_privileged = outcome_privileged[0] / (outcome_privileged[0] + outcome_privileged[3])
    tpr_unprivileged = outcome_unprivileged[0] / (outcome_unprivileged[0] + outcome_unprivileged[3])

    fairness = tpr_unprivileged - tpr_privileged
    
    return fairness

In [ ]:
def avg_odds_diff(input_data, output_data, attribute_name, privileged, unprivileged, favourable, unfavourable):
    outcome_both = output_rates(input_data, output_data, attribute_name, privileged, unprivileged, favourable, unfavourable)
    
    # [tp, fp, tn, fn]
    outcome_privileged = rates_both[0]
    outcome_unprivileged = rates_both[1]
    
    # true positive rate = tp / (tp + fn)
    tpr_privileged = outcome_privileged[0] / (outcome_privileged[0] + outcome_privileged[3])
    tpr_unprivileged = outcome_unprivileged[0] / (outcome_unprivileged[0] + outcome_unprivileged[3])

    # false positive rate = fp / (fp + tn)
    fpr_privileged = outcome_privileged[1] / (outcome_privileged[1] + outcome_privileged[2])
    fpr_unprivileged = outcome_unprivileged[1] / (outcome_unprivileged[1] + outcome_unprivileged[2])
    
    fpr_diff = fpr_unprivileged - fpr_privileged
    tpr_diff = tpr_unprivileged - tpr_unprivileged
    
    fairness = (fpr_diff + tpr_diff) * 0.5
    
    return fairness

In [157]:
def random_data():
    return pd.DataFrame(np.random.randint(0,2,size=(10, 5)), columns=list('ABCPO'))

In [161]:
for i in range(10):
    D = random_data()
    fairness = disparate_impact(D,'P',1,0,1,0)
    if fairness > 100:
        print("true")
        print(fairness)

{1: 3, 0: 1}
ratio_unpr:  0.75
{0: 4, 1: 2}
ratio_priv:  0.3333333333333333
{0: 5, 1: 2}
ratio_unpr:  0.2857142857142857
{1: 2, 0: 1}
ratio_priv:  0.6666666666666666
{0: 3}
{1: 3}
ratio_unpr:  1.0
{0: 4, 1: 3}
ratio_priv:  0.42857142857142855
{1: 5, 0: 2}
ratio_unpr:  0.7142857142857143
{1: 2, 0: 1}
ratio_priv:  0.6666666666666666
{1: 3, 0: 2}
ratio_unpr:  0.6
{0: 3, 1: 2}
ratio_priv:  0.4
{1: 4, 0: 3}
ratio_unpr:  0.5714285714285714
{1: 2, 0: 1}
ratio_priv:  0.6666666666666666
{1: 3, 0: 2}
ratio_unpr:  0.6
{0: 3, 1: 2}
ratio_priv:  0.4
{0: 3, 1: 2}
ratio_unpr:  0.4
{0: 3, 1: 2}
ratio_priv:  0.4
{0: 4, 1: 3}
ratio_unpr:  0.42857142857142855
{1: 2, 0: 1}
ratio_priv:  0.6666666666666666


In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.metrics import accuracy_score
from scipy.stats import mode
import pandas as pd
import numpy as np
import errno
import copy
from copy import deepcopy
import csv
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
def preprocess_compasdataset(df):
    df = df[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text',
                 'sex', 'priors_count', 'days_b_screening_arrest', 'decile_score',
                 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']]

    # Indices of data samples to keep
    ix = df['days_b_screening_arrest'] <= 30
    ix = (df['days_b_screening_arrest'] >= -30) & ix
    ix = (df['is_recid'] != -1) & ix
    ix = (df['c_charge_degree'] != "O") & ix
    ix = (df['score_text'] != 'N/A') & ix
    df = df.loc[ix,:]
    df['length_of_stay'] = (pd.to_datetime(df['c_jail_out']) - pd.to_datetime(df['c_jail_in'])).apply(lambda x: x.days)

    # Restrict races to African-American and Caucasian
    df = df.loc[~df['race'].isin(['Native American','Hispanic','Asian','Other']),:]

    df = df[['sex','race','age_cat','c_charge_degree','score_text','priors_count','is_recid', 'two_year_recid','length_of_stay']]

    df['priors_count'] = df['priors_count'].apply(lambda x: 0 if x <= 0 else ('1 to 3' if 1 <= x <= 3 else 'More than 3'))
    df['length_of_stay'] = df['length_of_stay'].apply(lambda x: '<week' if x <= 7 else ('<3months' if 8 < x <= 93 else '>3months'))
    df['score_text'] = df['score_text'].apply(lambda x: 'MediumHigh' if (x == 'High')| (x == 'Medium') else x)
    df['age_cat'] = df['age_cat'].apply(lambda x: '25 to 45' if x == '25 - 45' else x)

    df['sex'] = df['sex'].replace({'Female': 1.0, 'Male': 0.0})
    df['race'] = df['race'].apply(lambda x: 1.0 if x == 'Caucasian' else 0.0)

    df = df[['two_year_recid', 'sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']]

    protected_attributes = ['sex', 'race']
    label_name = 'two_year_recid'
    categorical_features = ['age_cat', 'priors_count', 'c_charge_degree']
    features = categorical_features + [label_name] + protected_attributes

    # privileged classes
    privileged_classes = {"sex": [1.0], "race": [1.0]}

    # protected attribute maps
    protected_attribute_map = {"sex": {0.0: 'Male', 1.0: 'Female'},
                                "race": {1.0: 'Caucasian', 0.0: 'Not Caucasian'}}


    data = StandardDataset(df, label_name, favorable_classes=[0],
                           protected_attribute_names=protected_attributes,
                           privileged_classes=[privileged_classes[x] for x in protected_attributes],
                           categorical_features=categorical_features,
                           features_to_keep=features,
                           metadata={'label_maps': [{1.0: 'Did recid.', 0.0: 'No recid.'}],
                                     'protected_attribute_maps': [protected_attribute_map[x] for x in protected_attributes]})

    return data

############ Reweighing ##############

def reweighing_data(train, unprivileged_group, privileged_group):
    RW = Reweighing(unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
    RW.fit(train)
    train_transformed = RW.transform(train)
    
    train_set = train_transformed

    # change weights to whole numbers
    for i in range(train_transformed.instance_weights.size):
        train_transformed.instance_weights[i] = (round(train_transformed.instance_weights[i] / 0.1) * 0.1) * 10
        weights = copy.deepcopy(train_transformed.instance_weights)

    # change train_transformed.features and train_transformed.labels and train_transformed.protected_attributes according to the weights of each instance
    for i in range(train_transformed.features.shape[0]):
        row = copy.deepcopy(train_transformed.features[i])
        row_label = copy.deepcopy(train_transformed.labels[i])
        row_protected_attributes = copy.deepcopy(train_transformed.protected_attributes[i])
        row_protected_attributes.resize(1,2)
        row.resize(1,train_transformed.features.shape[1])
        row_label.resize(1,1)
        weight = int(weights[i])
        for j in range(weight-1):
            train_transformed.features = np.concatenate((train_transformed.features,row))
            train_transformed.labels = np.concatenate((train_transformed.labels,row_label))
            train_transformed.protected_attributes = np.concatenate((train_transformed.protected_attributes,row_protected_attributes))

    # change the train_transformed to a numpy array of ones to match number of rows in features
    train_transformed.instance_weights = np.ones(train_transformed.features.shape[0])

    print("reweighing complete\n")

    return train_set, train_transformed

In [ ]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

print("Classification with Compas data set")
df = pd.read_csv('dataset/compas-scores-two-years.csv')
dataset_orig = preprocess_compasdataset(df)

train, test = dataset_orig.split([0.7], shuffle=True)

train_transformed, dataset_transf_train = reweighing_data(train, unprivileged_groups, privileged_groups)

 ##################### prejudice remover #####################
prejudice_model_reweighing = PrejudiceRemover(eta=100, sensitive_attr='sex')
prejudice_model_reweighing.fit(dataset_transf_train)
dataset_prejudice_test_reweighing = prejudice_model_reweighing.predict(test)

print("Prejudice Remover")

##################### metrics #####################

metric_test = BinaryLabelDatasetMetric(dataset_prejudice_test_reweighing,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

acc_test = ClassificationMetric(test, dataset_prejudice_test_reweighing,
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)

accuracy_prejudice = accuracy_score(y_true=test.labels, y_pred=dataset_prejudice_test_reweighing.labels)
# print('accuracy score = {}'.format(accuracy_prejudice))

print(acc_test.equal_opportunity_difference())
print(acc_test.average_odds_difference())
    
metrics_prejudice = [metric_test.mean_difference(), acc_test.disparate_impact(), np.nan, np.nan, acc_test.theil_index()]

##################### metrics #####################

# metric_test = BinaryLabelDatasetMetric(dataset_prejudice_test_reweighing,
#                                        unprivileged_groups=unprivileged_groups,
#                                        privileged_groups=privileged_groups)
# acc_test = ClassificationMetric(test, dataset_prejudice_test_reweighing,
#                                 unprivileged_groups=unprivileged_groups,
#                                 privileged_groups=privileged_groups)
# acc_test = BinaryLabelDatasetMetric(dataset_prejudice_test_reweighing,
#                                 unprivileged_groups=unprivileged_groups,
#                                 privileged_groups=privileged_groups)
# accuracy_prejudice = accuracy_score(y_true=test.labels, y_pred=dataset_prejudice_test_reweighing.labels)

#     equal_odds_difference = equal_opp_diff(test_df, datatest_df, 'sex', 1, 0, 1, 0)
#     equal_odds_difference = equal_opp_diff(test, dataset_prejudice_test_reweighing, 'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)

# metrics_prejudice = [metric_test.mean_difference(), acc_test.disparate_impact(), np.nan, np.nan, acc_test.theil_index()]
#     metrics_prejudice = [metric_test.mean_difference(), acc_test.disparate_impact(), equal_odds_difference, np.nan, acc_test.theil_index()]

Classification with Compas data set
reweighing complete

